In [1]:
import pandas as pd
def load_csv_from_github(file_name):
 base_url = "https://raw.githubusercontent.com/marymorkos/SalesPlaybookDS5640/refs/heads/main/"
 return pd.read_csv(base_url + file_name)

In [2]:
companies_df = load_csv_from_github("anonymized_hubspot_companies.csv")
companies_df.head()

C:\Users\laura\AppData\Local\Temp\ipykernel_13800\2943978199.py:4: DtypeWarning: Columns (24,25,28,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(base_url + file_name)


,Parent Company,CCaaS,Annual Revenue,Target Account,Associated Contact,Number of Form Submissions,Total Agents,Web Technologies,Close Date,# of Agents Total,...,BPO Program,Primary Sub-Industry,Number of Sessions,WFM,Country/Region,Industry,Create Date,Company name,Last Modified Date,BPO Program Tier
0,NaN,NaN,1.000000e+09,NaN,Contact_ef780380,0.0,NaN,Route 53,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,HVAC and plumbing supply,2024-10-30 10:51,Company_4fc73a2a,2025-02-14 13:39,NaN
1,NaN,NaN,NaN,NaN,Contact_93373ba5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pest Control,2024-09-12 18:30,Company_f3f7e884,2025-02-14 13:39,NaN
2,NaN,NaN,5.000000e+07,NaN,Contact_2e8e0993,0.0,NaN,Youtube; App Nexus; Google Tag Manager; Facebo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-09-03 10:44,Company_89929bed,2025-02-14 13:39,NaN
3,NaN,Five9,5.000000e+07,NaN,Contact_635e44ed,0.0,NaN,Postmark; Facebook Advertiser; Salesforce; Goo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_33dbf591,2025-02-14 13:39,NaN
4,NaN,Five9,1.000000e+08,NaN,Contact_8d055096,0.0,NaN,Microsoft Office 365; Google Tag Manager; Reca...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_a3079821,2025-02-14 13:39,NaN


In [ ]:
# load in deals merged data for mod
df = load_csv_from_github("deals_centric_merged.csv")
df.head()

,Weighted amount,Deal Description,"Cumulative time in ""BANT Deal. Pain ID'ed (Sales Pipeline)"" (HH:mm:ss)","Cumulative time in ""Opportunity (Sales Pipeline)"" (HH:mm:ss)",Days to close,Deal Score,Close Date,Deal source attribution 2,"Cumulative time in ""In Trial - Trial in Progress (Sales Pipeline)"" (HH:mm:ss)",Contract Start Date,...,Latest Milestone Update Date,Ticket name,Trial Overview,Trial Start Date,1st Syms presented for review,Project Launch Day,Training: Reporting,Construction of 1st Sym begun,Trial Required,Was the sym QAed?
0,250.0,40-50 employees,NaN,NaN,69.0,33.0,2025-05-01 15:26,Event,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,"3,000 + Agents.",NaN,NaN,149.0,13.0,2025-07-20 09:06,Referral Partner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,5-6k agents,NaN,NaN,149.0,13.0,2025-07-20 09:03,Referral Partner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Furthest Along - 300 agents,NaN,NaN,149.0,13.0,2025-07-20 08:55,Referral Partner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000.0,BPO through partnership with AmplifAI,NaN,NaN,120.0,37.0,2025-06-20 15:01,Master Agent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Find columns with less than 90% missing values 
columns_to_keep = missing_percentage[missing_percentage < 0.7].index

# Create a new dataframe with only the columns to keep
df_cleaned = df[columns_to_keep]

In [19]:
# Calculate percentage of missing values in each column
missing_percentage = df.isna().mean()

# Identify columns with 30-70% missing values
columns_30_70_missing = missing_percentage[(missing_percentage >= 0.3) & (missing_percentage <= 0.7)]

# Print the columns and their missing percentages
print("Columns with 30-70% missing values:")
for column, percent in columns_30_70_missing.items():
    print(f"{column}: {percent:.2%} missing")


Columns with 30-70% missing values:
Cumulative time in "BANT Deal. Pain ID'ed (Sales Pipeline)" (HH:mm:ss): 64.76% missing
Last Activity Date: 32.21% missing
Close Date_company: 54.81% missing
ICP Fit Level: 42.66% missing
Segmentation: 63.91% missing
Type: 32.38% missing
Primary Sub-Industry: 45.87% missing


In [30]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import re

def impute_sales_data(df, high_missing_threshold=0.3):
    """
    Efficient imputation pipeline for CRM sales data, handling both 
    high-missing-rate columns (30-70%) and low-missing-rate columns (<30%).
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with missing values
    high_missing_threshold (float): Threshold to determine which columns need advanced imputation
    
    Returns:
    pandas.DataFrame: DataFrame with all missing values imputed
    """
    print("Starting imputation pipeline...")
    result_df = df.copy()
    
    # Check if there are any missing values at all
    if not df.isna().any().any():
        print("No missing values found in dataset. No imputation needed.")
        return result_df
    
    # Classify columns by data type once for the whole process
    date_cols = [col for col in df.columns if 'Date' in col or 'date' in col]
    # Expanded time column detection
    time_cols = [col for col in df.columns if any(term in col.lower() for term in 
                ['time', 'HH:mm', 'duration', 'hours', 'minutes'])]
    
    # Special case for the known cumulative time column
    known_time_col = 'Cumulative time in "BANT Deal. Pain ID\'ed (Sales Pipeline)" (HH:mm:ss)'
    if known_time_col in df.columns and known_time_col not in time_cols:
        time_cols.append(known_time_col)
    
    # Ensure columns are unique across categories
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col not in date_cols and col not in time_cols]
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Identify high-missing and low-missing columns
    missing_rates = df.isna().mean()
    high_missing_cols = missing_rates[missing_rates >= high_missing_threshold].index.tolist()
    
    print(f"Identified column types: {len(date_cols)} date columns, {len(time_cols)} time columns, "
          f"{len(categorical_cols)} categorical columns, {len(numeric_cols)} numeric columns")
    print(f"Found {len(high_missing_cols)} columns with high missing rates (≥{high_missing_threshold*100}%)")
    
    # Process each column type with the appropriate method
    print("Processing date columns...")
    result_df = process_date_columns(result_df, date_cols, high_missing_cols, high_missing_threshold)
    
    print("Processing time columns...")
    result_df = process_time_columns(result_df, time_cols, high_missing_cols)
    
    print("Processing categorical columns...")
    result_df = process_categorical_columns(result_df, categorical_cols, high_missing_cols)
    
    print("Processing numeric columns...")
    result_df = process_numeric_columns(result_df, numeric_cols, high_missing_cols, high_missing_threshold)
    
    # Verify no NaN values remain
    remaining_nans = result_df.isna().sum()
    columns_with_nans = remaining_nans[remaining_nans > 0]
    
    if not columns_with_nans.empty:
        print(f"Warning: {len(columns_with_nans)} columns still have missing values after imputation:")
        for col, count in columns_with_nans.items():
            print(f"  - {col}: {count} missing values")
        print("Performing final cleanup...")
        
        # Final fallback for any remaining NaN values
        for col in columns_with_nans.index:
            dtype = result_df[col].dtype
            if pd.api.types.is_numeric_dtype(dtype):
                result_df[col] = result_df[col].fillna(0)
            elif pd.api.types.is_datetime64_dtype(dtype):
                result_df[col] = result_df[col].fillna(pd.Timestamp('2020-01-01'))
            else:
                result_df[col] = result_df[col].fillna('Unknown')
    
    print("Imputation complete!")
    return result_df

def process_date_columns(df, date_cols, high_missing_cols, threshold):
    """Handle date columns based on their missing rate"""
    result_df = df.copy()
    
    for col in date_cols:
        if col not in df.columns or not df[col].isna().any():
            continue
        
        # Create a copy of the original column in case conversion fails
        original_values = result_df[col].copy()
        
        try:
            # Convert to datetime
            result_df[col] = pd.to_datetime(result_df[col], errors='coerce')
            
            # Check if conversion was successful
            if result_df[col].isna().all():
                print(f"Warning: Could not convert column {col} to datetime. Using original values.")
                result_df[col] = original_values
                continue
            
            if col in high_missing_cols:
                # For high missing rates, use a reference date and simple imputation
                reference_date = pd.Timestamp('2020-01-01')
                
                # For datetimes, use median of available dates
                valid_dates = result_df[col].dropna()
                if not valid_dates.empty:
                    median_date = valid_dates.median()
                    result_df[col] = result_df[col].fillna(median_date)
                else:
                    # If all dates are missing, use reference date
                    result_df[col] = result_df[col].fillna(reference_date)
            else:
                # For low missing rates, use forward/backward fill or most frequent
                missing_pct = df[col].isna().mean()
                if missing_pct > 0.05:  # More than 5% missing
                    # First try to sort by a related date column if possible
                    date_cols_sorted = sorted(date_cols, key=lambda x: df[x].isna().mean())
                    for sort_col in date_cols_sorted:
                        if sort_col != col and not df[sort_col].isna().all() and pd.api.types.is_datetime64_dtype(result_df[sort_col]):
                            try:
                                # Try sorting by this column to improve ffill/bfill
                                temp_df = result_df[[sort_col, col]].sort_values(sort_col)
                                temp_df[col] = temp_df[col].ffill().bfill()
                                result_df[col] = temp_df[col]
                                break
                            except:
                                continue
                    
                    # If no sorting worked, just fill without sorting
                    if result_df[col].isna().any():
                        result_df[col] = result_df[col].ffill().bfill()
                else:
                    # For very low missing rates, use most frequent date
                    most_freq_date = result_df[col].mode()[0]
                    result_df[col] = result_df[col].fillna(most_freq_date)
            
            # Convert back to original string format if needed
            if df[col].dtype == 'object':
                try:
                    has_time = original_values.astype(str).str.contains(':').any()
                    format_str = '%m/%d/%Y %H:%M' if has_time else '%m/%d/%Y'
                    result_df[col] = result_df[col].dt.strftime(format_str)
                except:
                    # If datetime formatting fails, keep as is
                    print(f"Warning: Could not format dates in {col} back to string. Keeping datetime format.")
        except Exception as e:
            print(f"Error processing date column {col}: {e}")
            # Restore original values if processing failed
            result_df[col] = original_values
    
    return result_df

def process_time_columns(df, time_cols, high_missing_cols):
    """Handle time columns (HH:MM:SS format)"""
    result_df = df.copy()
    
    for col in time_cols:
        if col not in df.columns or not df[col].isna().any():
            continue
        
        try:
            # Keep original values in case of error
            original_values = result_df[col].copy()
            
            # Convert time to seconds for processing, with error handling
            try:
                result_df[f'{col}_seconds'] = df[col].apply(
                    lambda x: sum(int(t) * m for t, m in zip(x.split(':'), [3600, 60, 1])) 
                    if isinstance(x, str) and re.match(r'\d+:\d+:\d+', x) else np.nan
                )
            except Exception as e:
                print(f"Warning: Error converting time column {col} to seconds: {e}")
                continue
            
            # Check if conversion was successful
            if result_df[f'{col}_seconds'].isna().all():
                print(f"Warning: Could not convert any values in {col} to seconds. Skipping.")
                # Clean up and continue to next column
                if f'{col}_seconds' in result_df.columns:
                    result_df = result_df.drop(columns=[f'{col}_seconds'])
                continue
            
            # For any column, use median imputation (simple and effective for time)
            median_seconds = result_df[f'{col}_seconds'].median()
            if pd.isna(median_seconds):  # If median is NaN, use 0
                median_seconds = 0
            
            result_df[f'{col}_seconds'] = result_df[f'{col}_seconds'].fillna(median_seconds)
            
            # Convert seconds back to HH:MM:SS format
            try:
                result_df[col] = result_df[f'{col}_seconds'].apply(
                    lambda x: f"{int(x // 3600):d}:{int((x % 3600) // 60):02d}:{int(x % 60):02d}" 
                    if not pd.isna(x) else ""
                )
            except Exception as e:
                print(f"Warning: Error converting seconds back to time format for {col}: {e}")
                result_df[col] = original_values
            
            # Drop the temporary seconds column
            if f'{col}_seconds' in result_df.columns:
                result_df = result_df.drop(columns=[f'{col}_seconds'])
                
        except Exception as e:
            print(f"Error processing time column {col}: {e}")
            # Make sure column still exists in result_df
            if col not in result_df.columns:
                result_df[col] = original_values
    
    return result_df

def process_categorical_columns(df, categorical_cols, high_missing_cols):
    """Handle categorical columns"""
    result_df = df.copy()
    
    # Handle industry columns as a special case for consistency
    industry_cols = [col for col in categorical_cols if 'Industry' in col]
    if len(industry_cols) > 1:
        # Try to fill missing values from other industry columns first
        for i, col1 in enumerate(industry_cols):
            for col2 in industry_cols[i+1:]:
                # Where col1 is null but col2 isn't, copy from col2
                mask = df[col1].isna() & df[col2].notna()
                result_df.loc[mask, col1] = df.loc[mask, col2]
                
                # And vice versa
                mask = df[col2].isna() & df[col1].notna()
                result_df.loc[mask, col2] = df.loc[mask, col1]
    
    # Now process remaining categorical columns
    for col in categorical_cols:
        if col not in df.columns or not df[col].isna().any():
            continue
            
        if col in high_missing_cols:
            # For high missing rates, determine approach based on cardinality
            unique_count = df[col].nunique()
            
            if unique_count > 10:
                # Many categories - mark as "Unknown"
                result_df[col] = result_df[col].fillna('Unknown')
            else:
                # Few categories - use most frequent value
                most_freq = df[col].mode()[0]
                result_df[col] = result_df[col].fillna(most_freq)
        else:
            # For low missing rates, use most frequent
            most_freq = df[col].mode()[0]
            result_df[col] = result_df[col].fillna(most_freq)
    
    return result_df

def process_numeric_columns(df, numeric_cols, high_missing_cols, threshold):
    """Handle numeric columns"""
    result_df = df.copy()
    
    # Split financial columns vs other numeric
    financial_cols = [col for col in numeric_cols if any(term in col.lower() for term in 
                    ['amount', 'revenue', 'forecast', 'weighted'])]
    other_numeric = [col for col in numeric_cols if col not in financial_cols]
    
    # Process financial columns
    for col in financial_cols:
        if col not in df.columns or not df[col].isna().any():
            continue
            
        # Use median for financial columns regardless of missing rate
        # (simpler and often effective for financial data)
        median_val = df[col].median()
        result_df[col] = result_df[col].fillna(median_val)
    
    # Group remaining numeric columns by missing rate
    high_missing_numeric = [col for col in other_numeric if col in high_missing_cols]
    low_missing_numeric = [col for col in other_numeric if col not in high_missing_cols]
    
    # For high missing numeric, use MICE if there are multiple columns, otherwise use median
    if high_missing_numeric:
        # Filter to columns that actually have missing values
        cols_with_missing = [col for col in high_missing_numeric if df[col].isna().any()]
        
        if len(cols_with_missing) > 1:  # Multiple columns - use MICE
            imputer = IterativeImputer(
                estimator=RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42),
                max_iter=5,  # Reduced iterations
                random_state=42,
                skip_complete=True
            )
            
            # Get only rows where at least one column has a value (not all missing)
            valid_rows = df[cols_with_missing].notna().any(axis=1)
            
            if valid_rows.any():
                # Create temporary dataframe with only columns to impute
                temp_df = df.loc[valid_rows, cols_with_missing].copy()
                
                # Apply imputation
                imputed_values = imputer.fit_transform(temp_df)
                
                # Update result dataframe for these columns and rows
                for i, col in enumerate(cols_with_missing):
                    result_df.loc[valid_rows, col] = imputed_values[:, i]
                
                # For remaining rows, use median
                for col in cols_with_missing:
                    result_df[col] = result_df[col].fillna(df[col].median())
            else:
                # If all rows are completely missing, use median
                for col in cols_with_missing:
                    result_df[col] = result_df[col].fillna(0)  # Default to 0
        else:
            # Single column - use median (faster and simpler)
            for col in cols_with_missing:
                result_df[col] = result_df[col].fillna(df[col].median())
    
    # For low missing numeric, use simple median imputation
    if low_missing_numeric:
        cols_with_missing = [col for col in low_missing_numeric if df[col].isna().any()]
        
        for col in cols_with_missing:
            result_df[col] = result_df[col].fillna(df[col].median())
    
    # Special case: web analytics columns should be 0 if missing
    web_cols = [col for col in numeric_cols if any(term in col for term in 
                ['Pageviews', 'Sessions', 'Submissions', 'contacted'])]
    
    for col in web_cols:
        if col in result_df.columns and result_df[col].isna().any():
            result_df[col] = result_df[col].fillna(0)
    
    return result_df
df_imputed = impute_sales_data(df_cleaned)

Starting imputation pipeline...
Identified column types: 8 date columns, 3 time columns, 23 categorical columns, 19 numeric columns
Found 7 columns with high missing rates (≥30.0%)
Processing date columns...
Processing time columns...
Processing categorical columns...
Processing numeric columns...
  - Time Zone: 36 missing values
  - Consolidated Industry: 22 missing values
Performing final cleanup...
Imputation complete!


C:\Users\laura\AppData\Local\Temp\ipykernel_13800\1179323607.py:103: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result_df[col] = pd.to_datetime(result_df[col], errors='coerce')


In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix, 
    classification_report,
    roc_auc_score,
    roc_curve
)
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

# SMOTE for handling class imbalance
from imblearn.over_sampling import SMOTE

def prepare_features(df):
    # Columns to exclude - including potentially leaky features
    exclude_columns = [
        'Is Closed Won',  # Target variable
        'Is Closed (numeric)',
        'Is closed lost',
        'Is Deal Closed?',
        'Record ID',
        'Record ID_company',
        'Close Date',
        'Last Modified Date',
        'Create Date',
        'Deal Name',
        'Company name',
        'Deal probability',  # Potential leakage
        'Weighted amount',  # Derived from probability
        'Weighted amount in company currency'  # Also likely leaky
    ]

    # Features for consideration
    candidate_features = [
        # Business Context
        'Days to close',
        'Number of Pageviews',
        'Number of Form Submissions',
        'Number of Sessions',
        'Number of times contacted',
        
        # Financial Indicators
        'Forecast amount',
        'Amount',
        'Amount in company currency',
        'Annual Revenue',
        
        # Company Attributes
        'ICP Fit Level',
        'Industry',
        'Primary Industry',
        'Number of Employees',
        'Year Founded',
        
        # Engagement Metrics
        'Deal source attribution 2',
        'Original Traffic Source',
        'Pipeline',
        'Deal Type',
        'Segmentation'
    ]

    # Filter and prepare features
    features = [col for col in candidate_features if col in df.columns]
    
    # Prepare data
    X = df[features]
    y = df['Is Closed Won']

    # One-hot encode categorical columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    X = pd.get_dummies(X, columns=categorical_cols)

    # Ensure target is numeric
    if y.dtype == 'object' or y.dtype == 'bool':
        y = y.map({True: 1, 'True': 1, False: 0, 'False': 0})

    return X, y

# Print target distribution
print("Target Variable Distribution:")
print(df_imputed['Is Closed Won'].value_counts(normalize=True))

# Prepare the data
X, y = prepare_features(df_imputed)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1. Baseline Gradient Boosting with SMOTE
print("\n=== Gradient Boosting with SMOTE ===")
# Create pipeline with SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Print class distribution after SMOTE
print("\nClass distribution after SMOTE:")
unique, counts = np.unique(y_train_resampled, return_counts=True)
print(dict(zip(unique, counts)))

# Gradient Boosting with SMOTE
gb_smote = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_smote = cross_val_score(
    gb_smote, 
    X_train_resampled, 
    y_train_resampled, 
    cv=cv, 
    scoring='f1'
)
print("\nCross-validation F1 Scores with SMOTE:")
print(cv_scores_smote)
print(f"Mean CV F1 Score: {cv_scores_smote.mean():.4f} (+/- {cv_scores_smote.std() * 2:.4f})")

# Fit and predict with SMOTE
gb_smote.fit(X_train_resampled, y_train_resampled)
y_pred_smote = gb_smote.predict(X_test)
y_pred_proba_smote = gb_smote.predict_proba(X_test)[:, 1]

# Evaluation
print("\nSMOTE Model Performance:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_smote))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_smote))

print("\nAdditional Metrics:")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba_smote):.4f}")

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': gb_smote.feature_importances_
})
print("\nTop 10 Most Important Features:")
print(feature_importance.sort_values('importance', ascending=False).head(10))

Target Variable Distribution:
Is Closed Won
False    0.733558
True     0.266442
Name: proportion, dtype: float64

=== Gradient Boosting with SMOTE ===

Class distribution after SMOTE:
{0: 348, 1: 348}

Cross-validation F1 Scores with SMOTE:
[0.86713287 0.87142857 0.9        0.8951049  0.82706767]
Mean CV F1 Score: 0.8721 (+/- 0.0519)

SMOTE Model Performance:
Confusion Matrix:
[[77 10]
 [ 6 26]]

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        87
           1       0.72      0.81      0.76        32

    accuracy                           0.87       119
   macro avg       0.82      0.85      0.84       119
weighted avg       0.87      0.87      0.87       119


Additional Metrics:
ROC AUC Score: 0.9088

Top 10 Most Important Features:
                                        feature  importance
1                           Number of Pageviews    0.226414
6                                        Amount    0.0